In [ ]:
import torch
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
)
from bert_util import bert_tokenize_data, tensor_train_test_split, train_bert_model, model_predict, get_data_loader
import pandas as pd
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
import numpy as np
from sklearn.metrics import classification_report

# Normal Hand-Labeled Dataset

In [2]:
normal_train_df = pd.read_json('../normal_train_dataset.json', lines=False)
normal_test_df = pd.read_json('../normal_test_dataset.json', lines=False)
normal_train_df

,turn,utterance,emotion,act,hat
0,3,"I'll take one, too.",happiness,inform,0
1,8,"You know, we are superior to other clothes com...",no_emotion,inform,3
2,5,"Her new boyfriend, right?",no_emotion,commissive,1
3,9,How about recommending him to use the storage ...,no_emotion,directive,4
4,1,"Oh, a bouquet of flowers. It's very kind of you.",surprise,commissive,1
...,...,...,...,...,...
808,0,I prefer potatoes to eggplants.,no_emotion,inform,0
809,0,"Mr. Smith, I would like to get right to the po...",no_emotion,question,1
810,4,Yeah?,no_emotion,question,1
811,0,I am so bored all day.,no_emotion,inform,0


# Baseline roBERTa over the Normal Hand-Labeled Dataset

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
token_ids, attention_masks = bert_tokenize_data(tokenizer, normal_train_df['utterance'].values)
train_dataloader, val_dataloader = tensor_train_test_split(torch.tensor(normal_train_df['hat'].values), token_ids, attention_masks, test_size=0.1)
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)
optimizer = AdamW(model.parameters(), lr=2e-5)


configs = {
        "epochs": 10,
        "clip_grad_norm": 1.0,
        "early_stopping_patience": 3,
        "early_stopping_delta": 0.1,
    }

num_training_steps = 10 * len(train_dataloader)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps)

model = train_bert_model(model, optimizer, scheduler, train_dataloader, val_dataloader, configs)

In [4]:
test_texts = normal_test_df['utterance'].values
serie = pd.Series(test_texts)
tids, amids = bert_tokenize_data(tokenizer, serie, max_length=64)
dl = get_data_loader(tids, amids, batch_size=5, shuffle=False)
preds, confidences = model_predict(model, dl)
labels_flat = normal_test_df['hat'].values.flatten()
accuracy = np.sum(preds == labels_flat) / len(labels_flat)

In [5]:
hat_map = {
    0: "red",
    1: "white",
    2: "black",
    3: "yellow",
    4: "green",
}
preds_array = np.array(preds)
print(classification_report(labels_flat, preds_array, target_names=list(hat_map.values())))

              precision    recall  f1-score   support

         red       0.48      0.38      0.42        40
       white       0.66      0.84      0.74       110
       black       1.00      0.09      0.16        23
      yellow       0.45      0.47      0.46        19
       green       0.18      0.17      0.17        12

    accuracy                           0.59       204
   macro avg       0.55      0.39      0.39       204
weighted avg       0.61      0.59      0.55       204



# Baseline roBERTa over the EDA Augmented Hand-Labeled Dataset

In [6]:
augmented_train_df = pd.read_json('../eda_train_dataset.json', lines=False)

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
token_ids, attention_masks = bert_tokenize_data(tokenizer, augmented_train_df['utterance'].values)
train_dataloader, val_dataloader = tensor_train_test_split(torch.tensor(augmented_train_df['hat'].values), token_ids, attention_masks, test_size=0.1)
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)
optimizer = AdamW(model.parameters(), lr=2e-5)


configs = {
        "epochs": 10,
        "clip_grad_norm": 1.0,
        "early_stopping_patience": 3,
        "early_stopping_delta": 0.1,
    }

num_training_steps = 10 * len(train_dataloader)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps)

model = train_bert_model(model, optimizer, scheduler, train_dataloader, val_dataloader, configs)

In [8]:
test_texts = normal_test_df['utterance'].values
serie = pd.Series(test_texts)
tids, amids = bert_tokenize_data(tokenizer, serie, max_length=64)
dl = get_data_loader(tids, amids, batch_size=5, shuffle=False)
preds, confidences = model_predict(model, dl)
labels_flat = normal_test_df['hat'].values.flatten()
accuracy = np.sum(preds == labels_flat) / len(labels_flat)
hat_map = {
    0: "red",
    1: "white",
    2: "black",
    3: "yellow",
    4: "green",
}
preds_array = np.array(preds)
print(classification_report(labels_flat, preds_array, target_names=list(hat_map.values())))

              precision    recall  f1-score   support

         red       0.49      0.47      0.48        40
       white       0.70      0.78      0.74       110
       black       0.50      0.26      0.34        23
      yellow       0.53      0.53      0.53        19
       green       0.25      0.25      0.25        12

    accuracy                           0.61       204
   macro avg       0.49      0.46      0.47       204
weighted avg       0.60      0.61      0.60       204



# Baseline roBERTa over the Automated Labeled Dataset

In [15]:
ald_train_df = pd.read_json('../ald_train_dataset.json', lines=False)
ald_test_df = pd.read_json('../ald_test_dataset.json', lines=False)

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
token_ids, attention_masks = bert_tokenize_data(tokenizer, ald_train_df['utterance'].values)
train_dataloader, val_dataloader = tensor_train_test_split(torch.tensor(ald_train_df['hat'].values), token_ids, attention_masks, test_size=0.1)
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)
optimizer = AdamW(model.parameters(), lr=2e-5)


configs = {
        "epochs": 10,
        "clip_grad_norm": 1.0,
        "early_stopping_patience": 3,
        "early_stopping_delta": 0.1,
    }

num_training_steps = 10 * len(train_dataloader)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps)

model = train_bert_model(model, optimizer, scheduler, train_dataloader, val_dataloader, configs)

In [17]:
test_texts = ald_test_df['utterance'].values
serie = pd.Series(test_texts)
tids, amids = bert_tokenize_data(tokenizer, serie, max_length=64)
dl = get_data_loader(tids, amids, batch_size=5, shuffle=False)
preds, confidences = model_predict(model, dl)
labels_flat = ald_test_df['hat'].values.flatten()
accuracy = np.sum(preds == labels_flat) / len(labels_flat)
hat_map = {
    0: "red",
    1: "white",
    2: "black",
    3: "yellow",
    4: "green",
}
preds_array = np.array(preds)
print(classification_report(labels_flat, preds_array, target_names=list(hat_map.values())))

              precision    recall  f1-score   support

         red       0.75      0.77      0.76       200
       white       0.42      0.39      0.41       200
       black       0.51      0.59      0.55       200
      yellow       0.50      0.67      0.57       200
       green       0.65      0.37      0.47       200

    accuracy                           0.56      1000
   macro avg       0.57      0.56      0.55      1000
weighted avg       0.57      0.56      0.55      1000



# Testing the model on the normal test dataset

In [18]:
test_texts = normal_test_df['utterance'].values
serie = pd.Series(test_texts)
tids, amids = bert_tokenize_data(tokenizer, serie, max_length=64)
dl = get_data_loader(tids, amids, batch_size=5, shuffle=False)
preds, confidences = model_predict(model, dl)
labels_flat = normal_test_df['hat'].values.flatten()
accuracy = np.sum(preds == labels_flat) / len(labels_flat)
hat_map = {
    0: "red",
    1: "white",
    2: "black",
    3: "yellow",
    4: "green",
}
preds_array = np.array(preds)
print(classification_report(labels_flat, preds_array, target_names=list(hat_map.values())))

              precision    recall  f1-score   support

         red       0.32      0.30      0.31        40
       white       0.69      0.47      0.56       110
       black       0.19      0.22      0.20        23
      yellow       0.21      0.58      0.31        19
       green       0.25      0.25      0.25        12

    accuracy                           0.41       204
   macro avg       0.33      0.36      0.33       204
weighted avg       0.49      0.41      0.43       204

